In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Anxiety_disorder/GSE60491'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Leukocyte gene expression variation as a function of Big 5 dimensions of human personality"
!Series_summary	"Individual differences in basal leukocyte gene expression profiles as a function of Big 5 personality dimensions"
!Series_overall_design	"Gene expression profiling was carried out on peripheral blood mononuclear cell RNA samples collected from 119 healthy adults measured for the 5 major dimensions of human personality (Neuroticism, Extraversion, Openness, Agreeableness, Conscientiousness) using the NEO-FFI 60-item personality inventory.  Personality measures are z-score standardized.  Analyses control for major demographic characteristics (age, sex, Caucasian vs Non-Caucasian race) as well as Body Mass Index (BMI), smoking (CigDay), alcohol consumption (AlcDay), and physical activity (ExerDay, hours per day).  Additional secondary analyses controlled for the presence of minor physical symptoms (MinorSymptom, e.g.,hayfever, headache), medica

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 2.1: Data Availability
# Trait 'Anxiety_disorder': Not explicitly available or inferred
trait_row = None

# Age: Available in key 0
age_row = 0

# Gender: Available in key 1 under 'male' attribute
gender_row = 1

# Step 2.3: Data Type Conversion
# For 'age' (continuous) and 'gender' (binary)

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except:
        return None

def convert_gender(value):
    try:
        gender_val = value.split(':')[1].strip()
        return 1 if gender_val == '1' else 0
    except:
        return None

# Trait conversion function (dummy function since trait_row is None)
def convert_trait(value):
    return None

# Save Metadata
save_cohort_info('GSE60491', './preprocessed/Anxiety_disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Anxiety_disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Anxiety_disorder/trait_data/GSE60491.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
